In [2]:

import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

#sia = SentimentIntensityAnalyzer()

In [3]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cmozk\AppData\Roaming\nltk_data...


In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

11 Sep 2020 https://www.businesstimes.com.sg/technology/eu-urges-facebook-google-twitter-to-do-more-against-fake-news
10 Sep 2020 https://www.businesstimes.com.sg/technology/facebook-google-twitter-urged-by-eu-to-do-more-against-fake-news
08 Sep 2020 https://www.businesstimes.com.sg/real-estate/facebook-campus-hotel-a-bet-on-comeback-for-business-travel
07 Sep 2020 https://www.businesstimes.com.sg/technology/facebook-plans-to-label-posts-more-aggressively-clegg-says
03 Sep 2020 https://www.businesstimes.com.sg/technology/facebook-bans-indian-ruling-party-politician-for-policy-violation
01 Sep 2020 https://www.businesstimes.com.sg/technology/facebook-threatens-to-stop-publishers-in-australia-share-local-news-if-regulation-becomes
31 Aug 2020 https://www.businesstimes.com.sg/technology/facebook-says-eu-data-demands-included-risks-to-staffs-families
29 Aug 2020 https://www.businesstimes.com.sg/technology/google-facebook-dump-hong-kong-cable-after-us-security-alarm
29 Aug 2020 https://www.

In [15]:
from __future__ import (absolute_import, division, print_function,
                            unicode_literals)

# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')

import backtrader as bt
import backtrader.indicators as btind
import datetime
import os.path
import sys

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment

class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)
        

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Strategy
    cerebro.addstrategy(SentimentStrat)

    # Data Feed
    data = bt.feeds.YahooFinanceData(
        dataname = 'FB',
        fromdate = earliest_date,
        todate = datetime.datetime(2020,9,11),
        reverse = False
    )
    
    cerebro.adddata(data)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()
        

Starting Portfolio Value: 100000.00
2020-05-12, Close, 210.10
None
2020-05-13, Close, 205.10
None
2020-05-14, Close, 206.81
-0.542
2020-05-15, Close, 210.88
-0.542
2020-05-18, Close, 213.19
-0.542
2020-05-19, Close, 216.88
-0.542
2020-05-20, Close, 229.97
-0.715
2020-05-21, Close, 231.39
0.718
2020-05-21, BUY CREATE, 231.39
2020-05-22, BUY EXECUTED, Price: 231.51, Cost: 2315.10, Comm 2.32
2020-05-22, Close, 234.91
0.84
2020-05-26, Close, 232.20
0.84
2020-05-27, Close, 229.14
0.84
2020-05-28, Close, 225.46
0.974
2020-05-29, Close, 225.09
0.974
2020-06-01, Close, 231.91
0.974
2020-06-02, Close, 232.72
0.974
2020-06-03, Close, 230.16
-0.891
2020-06-04, Close, 226.29
-0.024
2020-06-05, Close, 230.77
0.593
2020-06-08, Close, 231.40
0.593
2020-06-09, Close, 238.67
-0.542
2020-06-10, Close, 236.73
-0.542
2020-06-11, Close, 224.43
-0.962
2020-06-12, Close, 228.58
-0.962
2020-06-15, Close, 232.50
-0.962
2020-06-16, Close, 235.65
-0.867
2020-06-17, Close, 235.53
-0.867
2020-06-18, Close, 235.94


<IPython.core.display.Javascript object>